In [2]:
import bs4 as bs
import urllib.request
import requests
import nltk
import re
import string
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, LSTM, InputLayer, Bidirectional, TimeDistributed, Embedding, Activation
from keras.optimizers import Adam
from keras.layers import Embedding
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
import re
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences



In [3]:
def preprocess_data(url_list, file1, additional_non_title):
    final_title_clean = []
    not_a_title = []
    for i in url_list:
        page = requests.get(i)
        page_text = page.content
        soup = bs.BeautifulSoup(page_text,'lxml')
        for tag in soup.findAll('a'):  ## span for ebay - h1 for ebay title
            if tag.text == '':
                pass
            else:
                length = nltk.word_tokenize(tag.text)
                if len(length) > 2: 
                    final_title_clean.append(" ".join(length))
                else:
                    not_a_title.append(" ".join(length))

    ### Title reading files
    title = file1["title"].to_list()
    title_2 = [incom for incom in title if str(incom) != 'nan']
    title_3 = title_2
    title_4 = []
    for i in title_3:
        l1 = i.split()
        title_4.append(" ".join(l1).lower())

    title_labels = [1]*len(title_4)

    #### Not a title
    no_title_cleaned = [re.sub(r"[^a-zA-Z0-9]", " ", file) for file in not_a_title]
    no_title_final = []
    for i in no_title_cleaned:
        l1 = i.split()
        if len(l1) != 0:
            no_title_final.append(" ".join(l1).lower())
            
    no_title_final = [item for item in no_title_final if not item.isdigit()]
    no_title_final = list(set(no_title_final))
    no_title_final = no_title_final + additional_non_title
    no_title_labels = [0]*len(no_title_final)

    training_data = title_4 + no_title_final
    labels = title_labels + no_title_labels

    return training_data, labels

url_list = ["https://www.ebay.com/sch/i.html?_from=R40&_trksid=p2047675.m570.l1313&_nkw=iphone&_sacat=0","https://www.etsy.com/search?q=python&order=price_desc&view_type=gallery&ship_to=us&explicit=1&page=1",
           "https://picclick.com/?q=python+skeleton","https://www.ebid.net/uk/perl/main.cgi?mo=search&words=cobra+scale&categoryid=&fulltext=&type=keyword&type1=a&type2=a&countryid=&used=&price1=&price2=&sfeedback=&near_me=&pcode=&force_keyword=&adult=&categoryonly=&category2=&useronly=&useronly_username=&view_format=gallery&sort=2-1",
           "https://www.glacierwear.com/catalogsearch/result/index/?limit=24&&q=python+hide","https://www.ecrater.com/filter.php?cid=0&keywords=python","https://www.thetaxidermystore.com/catalogsearch/result/index/?cat=0&dir=desc&limit=60&order=sku&q=snake+pelt",
           "https://www.1stdibs.com/furniture/more-furniture-collectibles/collectibles-curiosities/taxidermy/style/black-forest/","http://www.rubylane.com/ni/search?q=boa&sort=pricehigh&style=3&cstyle=1ttp://www.auctionzip.com/cgi-bin/auctionsearch.cgi?txtsearchradius=0&txtsearchzip=&idxsearchcategory=0&txtsearchkeywords=boa&month=&year=&searchpath=head&newsearch=1",
           "https://www.skullsunlimited.com/pages/search-results-page?q=python+skeleton","https://www.canadianlisted.com/snake+skin/listing","https://www.freeads.co.uk/search?for=snake+carcass&within_miles=201&sortby=price&sortdir=desc&view=list&page=1#.v_urmv7l_7a",
           "https://www.boneroom.com/apps/search?q=python+meat","https://www.trademe.co.nz/a/search?search_string=cobra&sort_order=PriceDesc&user_region=100&bof=Ajxc0vpf","https://www.trocadero.com/directory/search.php?keyword=taxidermy&click_to_search_now.x=0&click_to_search_now.y=0&over=&upto=&period=all+periods&archinsearch=no&category=ALL+ITEMS&item=&numitem=15&listtype=thumb&fromtrocadero=1",
           "http://www.implora.com/searchresult.asp?searchimp=taxidermy&submit=Search","https://www.1stdibs.com/search/?currency=usd&price=[*%20TO%20999]&q=ground%20dove","https://picclick.com/Taxidermy-Real-10-1-2-in-313332775598.html"]

additional_non_title = ['etsy window', 'window performance mark window', 'etsy logo','dangle drop earrings','chain link bracelets','baby boys clothing','mother of the dresses','baby toddler toys','for pets and pet lovers','browse similar','sale price offers browse similar','next page','log in','register','gift cards','etsy blog','sell on etsy','teams','forums','affiliates',
    'etsy inc','policies','investors','careers','press','impact','help center','trust and safety','privacy settings','the etsy app',
    'instagram','facebook','pinterest','twitter','youtube','terms of use','privacy','interest based ads','learn more','cancel loading','save the dates','programs or services for with or learning college','place your ad here','dont an account sign up','visit virginia real estate,shipping and delivery time','tell us what you in new window or tab',
    'feedback about your ebay search experience in new window or tab','do not sell my personal information','real estate, skip to main content', 'universal currency converter']

file1 = pd.read_csv("C:/Users/Vishal Patil/Desktop/Data Science/Sem 4/independent study/Dataset/ads_w_title_url_price_image_des.csv")

training_data, labels = preprocess_data(url_list, file1, additional_non_title)

In [4]:
def create_training_data(training_data, labels):
    tokenizer = Tokenizer(num_words = 120000) #40000 words are used here
    tokenizer.fit_on_texts(training_data)

    sequences = tokenizer.texts_to_sequences(training_data)
    #print(sequences)
    vocab_size = len(tokenizer.word_index) + 1
    (train_title, test_title, train_labels, test_labels) = train_test_split(sequences, labels, test_size=0.2)

    train_labels = to_categorical(train_labels, dtype ="uint8")
    test_labels = to_categorical(test_labels, dtype ="uint8")

    train_sentences_X = pad_sequences(train_title, maxlen=20, padding='post')
    test_sentences_X = pad_sequences(test_title, maxlen=20, padding='post')
    seq_length = train_sentences_X.shape[1]
    
    return train_sentences_X, train_labels, tokenizer

train_sentences_X, train_labels, tokenizer = create_training_data(training_data, labels)


In [5]:
### Building model

def create_model():
    model = Sequential()
    model.add(Embedding(100000, 32))
    model.add(LSTM(units = 128,return_sequences=True))
    model.add(LSTM(units = 64))
    model.add(Dense(units = 2, activation = 'sigmoid'))
    model.summary()

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    # fit model
    
    return model

model=create_model()
history = model.fit(train_sentences_X, train_labels,
                        batch_size=64,
                        epochs=2,
                        verbose=1,
                        validation_split=0.1)


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 32)          3200000   
_________________________________________________________________
lstm (LSTM)                  (None, None, 128)         82432     
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense (Dense)                (None, 2)                 130       
Total params: 3,331,970
Trainable params: 3,331,970
Non-trainable params: 0
_________________________________________________________________
Epoch 1/2
108/108 [==============================] - 14s 125ms/step - loss: 0.3403 - accuracy: 0.8985 - val_loss: 0.2025 - val_accuracy: 0.9161
Epoch 2/2
108/108 [==============================] - 11s 102ms/step - loss: 0.0651 - accuracy: 0.978

##  Test Websites

In [6]:

### Everythings perfect
#url = 'https://picclick.com/Crafts/Wholesale-Lots/?q=apple'
#url = 'https://picclick.com/Crafts/Wholesale-Lots/?q=mobile'
#url = 'https://picclick.com/?q=python+skeleton'  #- perfect extraction

#url = 'https://www.ecrater.com/c/895390/asian-antiques'
#url = 'https://www.ecrater.com/filter.php?cid=410974&keywords=baby+toys'
#url = 'https://www.ecrater.com/c/811711/books-manuscripts-antiques'
#url = 'https://www.ecrater.com/c/62/automotive'  #- perfect extraction
#url = 'https://www.ecrater.com/filter.php?keywords=automotive'

#url = 'https://www.ebid.net/uk/perl/main.cgi?go=1&mo=search&category=&type=keyword&words=baby&categoryid='
#url = 'https://www.ebid.net/uk/perl/main.cgi?go=1&mo=search&category=&type=keyword&words=glass&categoryid='
#url = 'https://www.ebid.net/uk/perl/main.cgi?mo=search&type=keyword&advanced_body=1&words=stamps&categoryid=&go=&price1=&price2=&countryid=&used=&sfeedback=&near_me=&pcode='
#url = 'https://www.ebid.net/uk/buy/clothes-shoes-accessories/baby/baby-accessories/'
#url = 'https://www.ebid.net/uk/buy/antiques/antiquities/'  #- perfect extraction
#url = 'https://www.ebid.net/uk/perl/main.cgi?mo=subcategory&type1=a&type2=a&categoryid=23&view_format=showroom&sort=0-0'

#url = 'https://www.ebay.com/sch/i.html?_from=R40&_trksid=m570.l1313&_nkw=cobra&_sacat=0'
#url = 'https://www.ebay.com/sch/i.html?_from=R40&_trksid=p2334524.m570.l2632&_nkw=mens+watches&_sacat=10290&LH_TitleDesc=0&_osacat=1059&_odkw=t-shirts'
#url = 'https://www.ebay.com/sch/i.html?_from=R40&_trksid=p2380057.m570.l2632&_nkw=handbags&_sacat=169291'
#url = 'http://www.ebay.com/sch/i.html?_from=r40&_sacat=0&_sop=16&_nkw=sharavati+loach&_ipg=200&rt=nc' #- perfect extraction
#url = 'https://www.ebay.com/sch/i.html?_from=r40&_sacat=0&_nkw=cobra&rt=nc'     #- perfect extraction
#url = 'https://www.ebay.com/sch/i.html?_from=R40&_trksid=p2334524.m570.l1313&_nkw=mobile&_sacat=0&LH_TitleDesc=0&_sop=16&_odkw=cobra&_osacat=0&rt=nc&_ipg=25' #- perfect extraction
#url = 'https://www.ebay.com/sch/i.html?_from=R40&_nkw=mobile&_sacat=0&LH_TitleDesc=0&_sop=16&_ipg=48'  #- perfect extraction

#url = 'https://www.etsy.com/c/clothing-and-shoes?ref=catnav-10923' #- perfect extraction
#url = 'https://www.etsy.com/search?q=jackets%20for%20men&ref=auto-1'
#url = 'https://www.etsy.com/search?q=mens%20wallet&ref=auto-1'
#url = 'https://www.etsy.com/c/clothing/mens-clothing/jackets-and-coats?ref=catnav-10923' #- perfect extraction

#url = 'https://www.boneroom.com/apps/search?q=animal+bones'
#url = 'https://www.boneroom.com/apps/search?q=books'
#url = 'https://www.boneroom.com/apps/search?q=mammals'
#url = 'https://www.boneroom.com/apps/search?q=bones'
#url = 'https://www.boneroom.com/apps/search?q=python+meat'
#url = 'https://www.boneroom.com/store/c213/Everyday_Fossils.html'


## Replace url variable in the next cell by any of the url mentioned from above cell

In [59]:
def predict_titles(url, model, tokenizer):
    page = requests.get(url)
    page_text = page.content
    soup = bs.BeautifulSoup(page_text,'lxml')
    tag_text = []
    for tag in soup.findAll('a'):  
        if tag.text == '':
            pass
        else:
            tag_text.append(tag.text)
    encoded = tokenizer.texts_to_sequences(tag_text)
    padded_test = pad_sequences(encoded, maxlen=20, padding='post')
    next_md = model.predict(padded_test, verbose=1)
    y_classes = next_md.argmax(axis=-1)
      
    encoded = tokenizer.texts_to_sequences(tag_text)
    padded_test = pad_sequences(encoded, maxlen=20, padding='post')
    next_md = model.predict(padded_test, verbose=1)

    y_classes = next_md.argmax(axis=-1)
    #print(total)
    reverse_word_map = dict(map(reversed, tokenizer.word_index.items()))

    return tag_text, y_classes, encoded, reverse_word_map

#url = 'https://picclick.com/Baby/?q=python+skeleton'
url = 'https://www.etsy.com/c/clothing-and-shoes?ref=catnav-10923'
#url = 'https://www.boneroom.com/apps/search?q=animal+bones'
tag_text, y_classes, encoded, reverse_word_map = predict_titles(url, model, tokenizer)

14/14 [==============================] - 0s 10ms/step


In [60]:
def sequence_to_text(list_of_indices):
    # Looking up words in dictionary
    words = [reverse_word_map.get(letter) for letter in list_of_indices]
    return(words)

# Creating texts 
my_texts = list(map(sequence_to_text, encoded))


In [61]:
def avg_length(y_classes, my_texts):
    index = []
    for i in range(len(y_classes)):
        if y_classes[i] == 1:
            index.append(" ".join(my_texts[i]))
    total = 0
    total_length = 0
    avg = sum(len(x.split()) for x in index) / len(index)
    #print(avg)
    extracted_titles = []
    for i in tag_text:
        i = i.split()
        if len(i) > avg:
            if not (re.search("Items|filter|feedback|think|Select", " ".join(i), flags=re.IGNORECASE)):
                total_length = total_length + len(i)
                total = total + 1
                extracted_titles.append(" ".join(i))
    return avg, extracted_titles

avg, extracted_titles = avg_length(y_classes, my_texts)

In [62]:
def first_extract_titles(url, avg, extracted_titles):
    page = requests.get(url)
    page_text = page.content
    soup = bs.BeautifulSoup(page_text,'lxml')
    s1 = soup.get_text("\n"," ")
    s2 = s1.split("\n")
    s3 = []
    for i in s2:
        s3.append(" ".join(i.split()))
    tot = 0
    final_titles_extract = []
    final_price_extracted = []
    for i in range(len(s3)):
        try:
            if s3[i] in extracted_titles and s3[i+1][0] == '$' or s3[i+1][0] == '£': 
                tot = tot + 1
                final_titles_extract.append(s3[i])
                final_price_extracted.append(s3[i+1])
            elif s3[i] in extracted_titles and s3[i+2][0] == '$' or s3[i+1][0] == '£':
                tot = tot + 1
                final_titles_extract.append(s3[i])
                final_price_extracted.append(s3[i+2])
            elif s3[i] in extracted_titles and s3[i+1][0] == '$'or s3[i+1][0] == '£': 
                tot = tot + 1
                final_titles_extract.append(s3[i])
                final_price_extracted.append(s3[i+1])
            elif s3[i] in extracted_titles and s3[i+2][0] == '$'or s3[i+1][0] == '£':
                tot = tot + 1
                final_titles_extract.append(s3[i])
                final_price_extracted.append(s3[i+2]) 
            elif s3[i][0] == '-' and len(s3[i].split()) > avg:
                final_titles_extract.append(s3[i])
                final_price_extracted.append(s3[i+1])
                tot = tot + 1
        except:
            pass
    
    avg_round = round(avg,0)
    for i in range(len(s2)):
        try:
            if s2[i][0] == '$' and len(s2[i-1].split()) == (avg_round - 1): 
                try:
                    float(s2[i][1:].replace(',',''))
                    final_titles_extract.append(s2[i-1])
                    final_price_extracted.append(s2[i])
                except:
                    pass
            elif s2[i][0] == '$' and len(s2[i-1].split()) == (avg_round - 2):
                try:
                    float(s2[i][1:].replace(',',''))
                    final_titles_extract.append(s2[i-1])
                    final_price_extracted.append(s2[i])            
                except:
                    pass
            elif s2[i][0] == '$' and len(s2[i-2].split()) == (avg_round - 1):
                try:
                    float(s2[i][1:].replace(',',''))
                    final_titles_extract.append(s2[i-2])
                    final_price_extracted.append(s2[i])
                except:
                    pass
            elif s2[i][0] == '$' and len(s2[i-2].split()) == (avg_round - 2):
                try:
                    float(s2[i][1:].replace(',',''))
                    final_titles_extract.append(s2[i-2])
                    final_price_extracted.append(s2[i])
                except:
                    pass
            elif s2[i][0] == '$' and len(s2[i-1].split()) == avg_round:     ### can remove this elif block only  
                try:                                                        ### not to overfit and extract all titles
                    float(s2[i][1:].replace(',',''))
                    final_titles_extract.append(s2[i-1])
                    final_price_extracted.append(s2[i])
                except:
                    pass
        except:
            pass
    final_titles_extract_updated= []
    for i in final_titles_extract:
        j = i.split()
        if j[0] == '-':
            tmp = j[1:]
            final_titles_extract_updated.append(" ".join(tmp))
        else:
            final_titles_extract_updated.append(" ".join(j))
    
    return final_titles_extract_updated, final_price_extracted

final_titles_extract_updated, final_price_extracted = first_extract_titles(url, avg, extracted_titles)

## Titles and Prices

In [63]:
def title_prices(final_titles_extract_updated, final_price_extracted, extracted_titles):
    title_price_df = pd.DataFrame()
    
    if len(final_price_extracted) > 1:
        dataframe = pd.DataFrame(zip(final_titles_extract_updated,final_price_extracted),columns = ['Title','Price'])
        title_price_df = title_price_df.append(dataframe)
    else:
        try:
            prices = []
            for i in extracted_titles:
                prices.append([x for x in re.findall(r"\$[^ ]+", i)])  #(\$[0-9]+(\.[0-9]$)?) # (\$[0-9]+(\.[0-9]$)?)
            dataframe_2 = pd.DataFrame(zip(extracted_titles,prices),columns = ['Title','Price'])#,'Image_url'])
            title_price_df = title_price_df.append(dataframe_2)
        except:
            pass
    
    return title_price_df

title_price_df = title_prices(final_titles_extract_updated, final_price_extracted, extracted_titles)

In [64]:
print(title_price_df)

                                                Title  \
0   More colors 39 Years of Being Awesome 39th Bir...   
1   More colors Caduceus Medical Rn, LPN, LVN, CNA...   
2   Graduation Gift - Class of - Graduation Keycha...   
3   Personalized Valentine's Day Gift for Daddy, P...   
4   More colors The shocker tshirt, sexual humor, ...   
5   Baby shoes for girls – Baby barefoot sandals w...   
6   More colors Custom handmade Gold or red Lace B...   
7   Son | Child | Dad | Original | Remix | Set | C...   
8   More colors Baby Girl Coming Home Outfit | New...   
9   More colors Thanksgiving shirt, women's Thanks...   
10  More colors Catholic Priest Gifts - Gift for W...   
11  Monogrammed pullover, personalized quarter zip...   
12  Leather Bag, Leather Satchel, Leather Messenge...   
13  More colors Octopus Playing Drums Men's T-Shir...   
14  More colors Personalized camera strap. Wonderf...   
15  Personalized leather Wallet, Personalized wall...   
16  More colors Purple Cheongsa

## Adding Urls to the existing dataframe

In [65]:
def extract_urls(url):
    page = requests.get(url)
    page_text = page.content
    soup = bs.BeautifulSoup(page_text,'lxml')
    links = []
    for link in soup.find_all('img'):
        links.append((link.get('alt'),link.get('src')))

    urls = []
    urls_2 = []
    for i in links:
        if i[0] in final_titles_extract_updated:
            urls.append(i[1])
        elif i[0] in extracted_titles:
            urls_2.append(i[1])
    if len(urls) == 0 and len(urls_2) == 0:
        titles_a_tag = []
        for tag in soup.find_all('a'):
            titles_a_tag.append((" ".join(tag.text.split()),tag))
        for i in titles_a_tag:
            if i[0] in final_titles_extract_updated:
                try:
                    for link in i[1].find_all('img'):
                        urls.append(link['src'])
                except:
                    for link in i[1].find_all('img'):
                        urls.append(link['data-src'])
                else:
                    try:
                        for link in i[1].find_all('div'):
                            urls.append(link['style'])
                    except:
                        pass
            elif i[0] in extracted_titles:
                try:
                    for link in i[1].find_all('img'):
                        urls_2.append(link['src'])                    
                except:
                    for link in i[1].find_all('img'):
                        urls_2.append(link['data-src'])
                else:
                    try:
                        for link in i[1].find_all('div'):
                            urls_2.append(link['style'])
                    except:
                        pass
    else:
        pass
    
    return urls, urls_2

urls, urls_2 = extract_urls(url)


In [66]:
def title_prices_urls(final_titles_extract_updated, final_price_extracted, extracted_titles, urls, urls_2):
    title_price_urls_df = pd.DataFrame()
    
    if len(final_price_extracted) > 1:
        dataframe = pd.DataFrame(zip(final_titles_extract_updated,final_price_extracted,urls),columns = ['Title','Price','Image_url'])
        title_price_urls_df = title_price_urls_df.append(dataframe)
    else:
        try:
            prices = []
            for i in extracted_titles:
                prices.append([x for x in re.findall(r"\$[^ ]+", i)])  #(\$[0-9]+(\.[0-9]$)?) # (\$[0-9]+(\.[0-9]$)?)
            dataframe_2 = pd.DataFrame(zip(extracted_titles,prices,urls_2),columns = ['Title','Price','Image_url'])#,'Image_url'])
            title_price_urls_df = title_price_urls_df.append(dataframe_2)
        except:
            pass
    
    return title_price_urls_df

title_price_urls_df = title_prices_urls(final_titles_extract_updated, final_price_extracted, extracted_titles, urls, urls_2)

In [67]:
print(title_price_urls_df)

                                                Title  \
0   More colors 39 Years of Being Awesome 39th Bir...   
1   More colors Caduceus Medical Rn, LPN, LVN, CNA...   
2   Graduation Gift - Class of - Graduation Keycha...   
3   Personalized Valentine's Day Gift for Daddy, P...   
4   More colors The shocker tshirt, sexual humor, ...   
5   Baby shoes for girls – Baby barefoot sandals w...   
6   More colors Custom handmade Gold or red Lace B...   
7   Son | Child | Dad | Original | Remix | Set | C...   
8   More colors Baby Girl Coming Home Outfit | New...   
9   More colors Thanksgiving shirt, women's Thanks...   
10  More colors Catholic Priest Gifts - Gift for W...   
11  Monogrammed pullover, personalized quarter zip...   
12  Leather Bag, Leather Satchel, Leather Messenge...   
13  More colors Octopus Playing Drums Men's T-Shir...   
14  More colors Personalized camera strap. Wonderf...   
15  Personalized leather Wallet, Personalized wall...   
16  More colors Purple Cheongsa